In [15]:
# 패키지 정의
import numpy as np
# Tokenizer : 문장에서 단어 분류하기
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [2]:
text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n
"""
print(text)
text

경마장에 있는 말이 뛰고 있다

그의 말이 법이다

가는 말이 고와야 오는 말이 곱다




'경마장에 있는 말이 뛰고 있다\n\n그의 말이 법이다\n\n가는 말이 고와야 오는 말이 곱다\n\n'

In [3]:
# 문장에서 단어별 딕셔너리 생성
# 딕셔너리 값은 각 단어의 고유 인덱스 값이 됨
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
print(tokenizer.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [4]:
### 단어 크기(갯수) 저장

vocab_size = len(tokenizer.word_index)+1
print("단어 인덱스 총 크기 : ", vocab_size)

단어 인덱스 총 크기 :  12


In [5]:
### 단어를 숫자값으로 변환하여 연속된 단어별로 조합하기
### - 모든 숫자의 조합 저장할 리스트 변수 정의
sequences = []

### 한글 문장을 줄바꿈(\n)을 기준으로 문장 나누기
for line in text.split("\n") :
    encoded = tokenizer.texts_to_sequences([line])[0]
    print(">",encoded)
    
    ### 문장 내 단어별 조합하기
    for i in range(1, len(encoded)) :
        ### 이전 단어의 인덱스 값을 연결하여 조합하기
        sequence = encoded[:i+1]
        print("->", encoded[:i+1])
        
        ### 리스트 변수에 담기
        sequences.append(sequence)
        print("-->>", sequences)
        
print("학습에 사용할 샘플의 개수 :", len(sequences))

> [2, 3, 1, 4, 5]
-> [2, 3]
-->> [[2, 3]]
-> [2, 3, 1]
-->> [[2, 3], [2, 3, 1]]
-> [2, 3, 1, 4]
-->> [[2, 3], [2, 3, 1], [2, 3, 1, 4]]
-> [2, 3, 1, 4, 5]
-->> [[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5]]
> []
> [6, 1, 7]
-> [6, 1]
-->> [[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1]]
-> [6, 1, 7]
-->> [[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7]]
> []
> [8, 1, 9, 10, 1, 11]
-> [8, 1]
-->> [[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1]]
-> [8, 1, 9]
-->> [[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9]]
-> [8, 1, 9, 10]
-->> [[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10]]
-> [8, 1, 9, 10, 1]
-->> [[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1]]
-> [8, 1, 9, 10, 1, 11]
-->> [[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8,

In [6]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [7]:
# 문장의 최대길이 찾기
max_len = max(len(i) for i in sequences)
print("샘플의 최대길이 찾기 : ", max_len)

샘플의 최대길이 찾기 :  6


In [8]:
# 길이를 최대길이로 통일 시키기
sequences = pad_sequences(sequences, maxlen=max_len,padding="pre")
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [9]:
# 훈련 입력 데이터 : x변수
# 훈련 타겟 데이터 : y변수
# 각 문장의 마지막 값을 y값으로 사용
# 배열로 만들기
# x : (11,5)
# y : (11,)
sequences = np.array(sequences)
x = sequences[:,:-1]
y = sequences[:,-1]
print(x.shape,y.shape)
print(x)
print(y)

(11, 5) (11,)
[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]
[ 3  1  4  5  1  7  1  9 10  1 11]


In [10]:
y = to_categorical(y,num_classes=vocab_size)
print(y.shape)
print(y)

(11, 12)
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

model = Sequential()

In [12]:
# vocab_size : 주요 단어 12개 
# 10 : 단어 임베딩 벡터(출력값)
model.add(Embedding(vocab_size, 10))
# 32 : RNN 출력값
model.add(SimpleRNN(32))
# 출력계층(종속변수 y의 개수만큼 출력)
model.add(Dense(vocab_size,activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 10)          120       
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                1376      
                                                                 
 dense (Dense)               (None, 12)                396       
                                                                 
Total params: 1,892
Trainable params: 1,892
Non-trainable params: 0
_________________________________________________________________


In [13]:
# 모델 설정 : 컴파일
model.compile(optimizer="adam",
             loss="categorical_crossentropy",
             metrics=["accuracy"])
# 모델 훈련
# verbose : 출력 형태 지정
model.fit(x,y,epochs=200,verbose=1)

Epoch 1/200
1/1 [==============================] - 1s 696ms/step - loss: 2.5087 - accuracy: 0.0000e+00
Epoch 2/200
1/1 [==============================] - 0s 10ms/step - loss: 2.4924 - accuracy: 0.0000e+00
Epoch 3/200
1/1 [==============================] - 0s 7ms/step - loss: 2.4764 - accuracy: 0.0000e+00
Epoch 4/200
1/1 [==============================] - 0s 5ms/step - loss: 2.4606 - accuracy: 0.0909
Epoch 5/200
1/1 [==============================] - 0s 4ms/step - loss: 2.4448 - accuracy: 0.2727
Epoch 6/200
1/1 [==============================] - 0s 4ms/step - loss: 2.4288 - accuracy: 0.3636
Epoch 7/200
1/1 [==============================] - 0s 3ms/step - loss: 2.4125 - accuracy: 0.3636
Epoch 8/200
1/1 [==============================] - 0s 5ms/step - loss: 2.3958 - accuracy: 0.3636
Epoch 9/200
1/1 [==============================] - 0s 4ms/step - loss: 2.3786 - accuracy: 0.3636
Epoch 10/200
1/1 [==============================] - 0s 4ms/step - loss: 2.3609 - accuracy: 0.3636
Epoch 11/200
1

1/1 [==============================] - 0s 3ms/step - loss: 0.9818 - accuracy: 0.8182
Epoch 85/200
1/1 [==============================] - 0s 3ms/step - loss: 0.9633 - accuracy: 0.9091
Epoch 86/200
1/1 [==============================] - 0s 3ms/step - loss: 0.9452 - accuracy: 0.8182
Epoch 87/200
1/1 [==============================] - 0s 4ms/step - loss: 0.9273 - accuracy: 0.8182
Epoch 88/200
1/1 [==============================] - 0s 2ms/step - loss: 0.9097 - accuracy: 0.8182
Epoch 89/200
1/1 [==============================] - 0s 3ms/step - loss: 0.8923 - accuracy: 0.8182
Epoch 90/200
1/1 [==============================] - 0s 4ms/step - loss: 0.8753 - accuracy: 0.8182
Epoch 91/200
1/1 [==============================] - 0s 3ms/step - loss: 0.8585 - accuracy: 0.8182
Epoch 92/200
1/1 [==============================] - 0s 2ms/step - loss: 0.8421 - accuracy: 0.8182
Epoch 93/200
1/1 [==============================] - 0s 3ms/step - loss: 0.8259 - accuracy: 0.8182
Epoch 94/200
1/1 [===============

1/1 [==============================] - 0s 2ms/step - loss: 0.2591 - accuracy: 1.0000
Epoch 168/200
1/1 [==============================] - 0s 3ms/step - loss: 0.2554 - accuracy: 1.0000
Epoch 169/200
1/1 [==============================] - 0s 2ms/step - loss: 0.2518 - accuracy: 1.0000
Epoch 170/200
1/1 [==============================] - 0s 2ms/step - loss: 0.2481 - accuracy: 1.0000
Epoch 171/200
1/1 [==============================] - 0s 3ms/step - loss: 0.2445 - accuracy: 1.0000
Epoch 172/200
1/1 [==============================] - 0s 4ms/step - loss: 0.2409 - accuracy: 1.0000
Epoch 173/200
1/1 [==============================] - 0s 3ms/step - loss: 0.2373 - accuracy: 1.0000
Epoch 174/200
1/1 [==============================] - 0s 2ms/step - loss: 0.2337 - accuracy: 1.0000
Epoch 175/200
1/1 [==============================] - 0s 2ms/step - loss: 0.2301 - accuracy: 1.0000
Epoch 176/200
1/1 [==============================] - 0s 3ms/step - loss: 0.2265 - accuracy: 1.0000
Epoch 177/200
1/1 [=====

In [16]:
# 모델 저장 및 불러오기

# 훈련 모델 저장하기
keras.models.save_model(model, "./model/django_rnn_model.h5")

In [ ]:
# 훈련 모델 불러오기
model = keras.models.load_model("./model/django_rnn_model.h5")

In [28]:
# 예측할 단어로부터 다음 단어를 예측하기위한 함수
# 매개변수 : 모델, 토큰나이저, 전달받은 단어, 반복할 횟수
def sentence_generator(model, tokenizer, current_word, n):
    init_word = current_word
    sentence = ""
    
    # 전달받은 값 n번 반복시키기
    for _ in range(n):
        # 현재입력 받은 단어에 대한 정수 인코딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        # 단어 자르기 : x값은 5개
        encoded = pad_sequences([encoded], maxlen=5, padding="pre")
        # 에측하기
        result = model.predict(encoded, verbose=0)
        # 예측결과 중 가장 빈도가 큰 값 결정
        result = np.argmax(result, axis=1)
        
        for word, index in tokenizer.word_index.items():
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
            if index == result:
                break
                
        # 현재 입력된 단어와 예측 단어 조합하기
        current_word = current_word + " " + word
        
        # 예측 단어 조합을 변수에 저장
        sentence = sentence + " " + word
    # 함수가 전달받은 단어에 대한 조합할 단어 합치기
    sentence = init_word + sentence
    
    return sentence

In [29]:
# 함수 실행하기
print(sentence_generator(model,tokenizer,"경마장에",4))

경마장에 있는 말이 뛰고 있다


In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 10)          120       
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                1376      
                                                                 
 dense (Dense)               (None, 12)                396       
                                                                 
Total params: 1,892
Trainable params: 1,892
Non-trainable params: 0
_________________________________________________________________
